In [1]:
import numpy as np

import cv2
import json
import warnings

warnings.simplefilter("ignore")

from skimage import draw

from pathlib import Path
from skimage import io
from tqdm.notebook import tqdm

from skimage import draw
from skimage.transform import resize
from skimage.morphology import binary_dilation
from matplotlib import pyplot as plt

SIZE = 384

# IMG_DIR_PATH = "../../data/raw_images/"
# CIRCLE_DIR_PATH = "../../manual_markup/raw_circles/"
# #ELLIPSE_DIR_PATH = "../../manual_markup/british_ellipses/"


# IMG_OUTPUT = "../../cropped_images/" + str(SIZE) + "/raw_images/"
# CIRCLE_OUTPUT = "../../cropped_images/" + str(SIZE) + "/raw_circle_masks/"
# #ELLIPSE_OUTPUT = "../../cropped_images/" + str(SIZE) + "/british_ellipse_masks/"

In [2]:
ratios = {}

In [3]:
IMG_PATH = "../../data/all_images/"
CIRCLE_PATH = "../../data/circles_1.0/"
ELLIPSE_PATH = "../../data/ellipses_3.2_1.7/"

OUT_IMG = "../../cropped_384/images/"
OUT_CIRCLE = "../../cropped_384/circles/"
OUT_ELLIPSE = "../../cropped_384/ellipses/"

In [4]:
coefs = {}
with open("coefs.json") as f:
    coefs = json.load(f)

In [5]:
shapes = []

for path in tqdm([*Path(CIRCLE_PATH).glob('*_spmk_1.0.png')]):
    name = str(path.stem)[:-9]
    img = io.imread(str(path))

    ##########################################
    xc, yc = np.where(img > 0)
    xs = np.min(xc) - int(0.05 * img.shape[0])
    xe = np.max(xc) + int(0.05 * img.shape[0])

    dx = xe - xs
    if xs < 0:
        xs = 0
        xe = dx
    if xe > img.shape[0]:
        xe = img.shape[0]
        xs = xe - dx

    ys = np.min(yc) + 0.5 * (np.max(yc) - np.min(yc)) - dx // 2
    ys = int(ys)
    if ys + xe -xs > img.shape[1]:
        ye = img.shape[1]
    else:
        ye = ys + xe -xs
    ys = ye - (xe -xs)

    if (xe - xs) != (ye - abs(ys)):
        print(name)
        print(xs, xe, ys, ye)
        ye += abs(ys)
        ys = 0
        print(xs, xe, ys, ye)
        print("\n")
        
    cropped = img[xs:xe, ys:ye]
    cropped[cropped > 0] = 255

    contours = cv2.findContours(cropped, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0] # cv2.CHAIN_APPROX_SIMPLE
    spikelets_centers = []
    for c in contours:
        M = cv2.moments(c)
        # центр масс
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        spikelets_centers.append([cy, cx])

    spikelets_centers = sorted(spikelets_centers, key=lambda p: p[0])
    mm_masks = np.zeros_like(cropped)
    radius = 1 * np.sqrt(coefs[name])
    for center in spikelets_centers:
        rr, cc = draw.disk(center=tuple(center), radius=radius, shape=mm_masks.shape)
        mm_masks[rr, cc] = 1
    
    shapes.append(cropped.shape[0])
    cropped = cv2.resize(mm_masks, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)
    cropped[cropped > 0] = 255
    cropped = cropped.astype(np.uint8)
    ##########################################

    io.imsave(OUT_CIRCLE + name + '.png', cropped)

    

    img = io.imread(ELLIPSE_PATH + name + '_spmkel_3.2_1.7.png')
    cropped = img[xs:xe, ys:ye]
    cropped = resize(cropped, (SIZE, SIZE)) * 255
    cropped = cropped.astype(np.uint8)
    io.imsave(OUT_ELLIPSE + name + '.png', cropped)

    

    if name[:5] == "Spike":
        img = io.imread(IMG_PATH + name + '.JPG')
    else:
        img = io.imread(IMG_PATH + name + '.jpg')
        
    cropped = img[xs:xe, ys:ye]
    if cropped.shape[0] != cropped.shape[1]:
        print(name)
        print(xs, xe, ys, ye)
        print(cropped.shape[0], cropped.shape[1])
        
    ratios[name] = (xe - xs) / SIZE
    cropped = resize(cropped, (SIZE, SIZE)) * 255
    cropped = cropped.astype(np.uint8)
    io.imsave(OUT_IMG + name + '.jpg', cropped)

  0%|          | 0/1701 [00:00<?, ?it/s]

Spike_664
0 1285 -171 1114
0 1285 0 1285


Spike_672
47 1376 -142 1187
47 1376 0 1329


Spike_535
18 1209 -214 977
18 1209 0 1191


Spike_670
0 1260 -2 1258
0 1260 0 1260


Spike_580
90 1207 -19 1098
90 1207 0 1117


Spike_755
140 1236 -36 1060
140 1236 0 1096


Spike_717
42 1218 -67 1109
42 1218 0 1176


Spike_579
41 1082 -40 1001
41 1082 0 1041


Spike_645
151 1251 -84 1016
151 1251 0 1100


Spike_793
159 1402 -53 1190
159 1402 0 1243


Spike_716
173 1464 -126 1165
173 1464 0 1291


Spike_715
132 1261 -44 1085
132 1261 0 1129


Spike_532
183 1331 -8 1140
183 1331 0 1148




In [6]:
import json

with open('ratios.json', 'w') as f:
    json.dump(ratios, f)